# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
Arxiv has 60 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.18006


extracting tarball to tmp_2311.18006...

 done.


S. Li  ->  S. Li  |  ['S. Li']


...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2311.18067/sample631.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'fancytiles' from 'tmp_2311.18067/fancytiles.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'NHsummary.tex' from 'tmp_2311.18067/NHsummary.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/a

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 125 bibliographic references in tmp_2311.18067/sample631.bbl.
syntax error in line 711: '=' expected
Retrieving document from  https://arxiv.org/e-print/2311.18321


extracting tarball to tmp_2311.18321...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.18006-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.18006) | **Digging into the Interior of Hot Cores with ALMA: Spiral Accretion into  the High-mass Protostellar Core G336.01-0.82**  |
|| F. Olguin, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-12-01*|
|*Comments*| *11 pages, 5 figures, 2 appendices. Accepted for publication in ApJL. Interactive figure available at this https URL*|
|**Abstract**| We observed the high-mass star-forming core G336.01-0.82 at 1.3 mm and 0.05'' (~150 au) angular resolution with the Atacama Large Millimeter/submillimeter Array (ALMA) as part of the Digging into the Interior of Hot Cores with ALMA (DIHCA) survey. These high-resolution observations reveal two spiral streamers feeding a circumstellar disk at opposite sides in great detail. Molecular line emission from CH$_3$OH shows velocity gradients along the streamers consistent with infall. Similarly, a flattened envelope model with rotation and infall implies a mass larger than 10 M$_\odot$ for the central source and a centrifugal barrier of 300 au. The location of the centrifugal barrier is consistent with local peaks in the continuum emission. We argue that gas brought by the spiral streamers is accumulating at the centrifugal barrier, which can result in future accretion burst events. A total high infall rate of ~$4\times10^{-4}$ M$_\odot$ yr$^{-1}$ is derived by matching models to the observed velocity gradient along the streamers. Their contribution account for 20-50% the global infall rate of the core, indicating streamers play an important role in the formation of high-mass stars. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.18016-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.18016) | **Preparing for low surface brightness science with the Vera C. Rubin  Observatory: A Comparison of Observable and Simulated Intracluster Light  Fractions**  |
|| S. Brough, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-12-01*|
|*Comments*| *Submitted for publication in MNRAS, posted to arXiv after responding to two positive rounds of referee comments. Key results in Figs 3, 5, 6 and 11*|
|**Abstract**| Intracluster Light (ICL) provides an important record of the interactions galaxy clusters have undergone. However, we are limited in our understanding by our measurement methods. To address this we measure the fraction of cluster light that is held in the Brightest Cluster Galaxy and ICL (BCG+ICL fraction) and the ICL alone (ICL fraction) using observational methods (Surface Brightness Threshold-SB, Non-Parametric Measure-NP, Composite Models-CM, Multi-Galaxy Fitting-MGF) and new approaches under development (Wavelet Decomposition-WD) applied to mock images of 61 galaxy clusters (14<log10 M_200c/M_solar <14.5) from four cosmological hydrodynamical simulations. We compare the BCG+ICL and ICL fractions from observational measures with those using simulated measures (aperture and kinematic separations). The ICL fractions measured by kinematic separation are significantly larger than observed fractions. We find the measurements are related and provide equations to estimate kinematic ICL fractions from observed fractions. The different observational techniques give consistent BCG+ICL and ICL fractions but are biased to underestimating the BCG+ICL and ICL fractions when compared with aperture simulation measures. Comparing the different methods and algorithms we find that the MGF algorithm is most consistent with the simulations, and CM and SB methods show the smallest projection effects for the BCG+ICL and ICL fractions respectively. The Ahad (CM), MGF and WD algorithms are best set up to process larger samples, however, the WD algorithm in its current form is susceptible to projection effects. We recommend that new algorithms using these methods are explored to analyse the massive samples that Rubin Observatory's Legacy Survey of Space and Time will provide. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.18067-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.18067) | **A Two-Component Probability Distribution Function Describes the mid-IR  Emission from the Disks of Star-Forming Galaxies**  |
|| D. Pathak, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-12-01*|
|*Comments*| *30 pages without appendix, 17 figures, (with appendix images of full sample: 56 pages, 39 figures), accepted in AJ*|
|**Abstract**| High-resolution JWST-MIRI images of nearby spiral galaxies reveal emission with complex substructures that trace dust heated both by massive young stars and the diffuse interstellar radiation field. We present high angular (0."85) and physical resolution (20-80 pc) measurements of the probability distribution function (PDF) of mid-infrared (mid-IR) emission (7.7-21 $\mu$m) from 19 nearby star-forming galaxies from the PHANGS-JWST Cycle-1 Treasury. The PDFs of mid-IR emission from the disks of all 19 galaxies consistently show two distinct components: an approximately log-normal distribution at lower intensities and a high-intensity power-law component. These two components only emerge once individual star-forming regions are resolved. Comparing with locations of HII regions identified from VLT/MUSE H$\alpha$-mapping, we infer that the power-law component arises from star-forming regions and thus primarily traces dust heated by young stars. In the continuum-dominated 21 $\mu$m band, the power-law is more prominent and contains roughly half of the total flux. At 7.7-11.3 $\mu$m, the power-law is suppressed by the destruction of small grains (including PAHs) close to HII regions while the log-normal component tracing the dust column in diffuse regions appears more prominent. The width and shape of the log-normal diffuse emission PDFs in galactic disks remain consistent across our sample, implying a log-normal gas column density $N$(H)$\approx10^{21}$cm$^{-2}$ shaped by supersonic turbulence with typical (isothermal) turbulent Mach numbers $\approx5-15$. Finally, we describe how the PDFs of galactic disks are assembled from dusty HII regions and diffuse gas, and discuss how the measured PDF parameters correlate with global properties such as star-formation rate and gas surface density. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.18321-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.18321) | **2D disc modelling of the JWST line spectrum of EX Lupi**  |
|| P. Woitke, et al. -- incl., <mark>A. Kospal</mark> |
|*Appeared on*| *2023-12-01*|
|*Comments*| *accepted by A&A, 25 pages, 15 figures, 7 tables*|
|**Abstract**| We introduce a number of new theoretical approaches and improvements to the thermo-chemical disc modelling code ProDiMo to better predict and analyse the JWST line spectra of protoplanetary discs. We develop a new line escape probability method for disc geometries, a new scheme for dust settling, and discuss how to apply UV molecular shielding factors to photorates in 2D disc geometry. We show that these assumptions are crucial for the determination of the gas heating/cooling rates and discuss how they affect the predicted molecular concentrations and line emissions. We apply our revised 2D models to the protoplanetary disc around the T Tauri star EX Lupi in quiescent state. We calculate infrared line emission spectra between 5 and 20 mic by CO, H2O, OH, CO2, HCN, C2H2 and H2, including lines of atoms and ions, using our full 2D predictions of molecular abundances, dust opacities, gas and dust temperatures. We develop a disc model with a slowly increasing surface density structure around the inner rim that can simultaneously fit the spectral energy distribution, the overall shape of the JWST spectrum of EX Lupi, and the main observed molecular characteristics in terms of column densities, emitting areas and molecular emission temperatures, which all result from one consistent disc model. The spatial structure of the line emitting regions of the different molecules is discussed. High abundances of HCN and C2H2 are caused in the model by stellar X-ray irradiation of the gas around the inner rim. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.18006.md
    + _build/html/tmp_2311.18006/./fig2.png
    + _build/html/tmp_2311.18006/./fig1.png
    + _build/html/tmp_2311.18006/./fig3.png
exported in  _build/html/2311.18016.md
    + _build/html/tmp_2311.18016/./Sims_BCG_clustermass.png
    + _build/html/tmp_2311.18016/./Sims_clustermass.png
    + _build/html/tmp_2311.18016/./ICLBCG_Frac_clustermass.png
    + _build/html/tmp_2311.18016/./ICLBCG_FracDiff_histo.png
    + _build/html/tmp_2311.18016/./ICL_Frac_clustermass_aper.png
exported in  _build/html/2311.18067.md
    + _build/html/tmp_2311.18067/./overalltiles_NGC4321.png
    + _build/html/tmp_2311.18067/./HII_PDFs_v0p9p2_atGauss0p85.png
    + _build/html/tmp_2311.18067/./HII_logPDFs_v0p9p2_atGauss0p85.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msun}{{\rm M}_\sun}$
$\newcommand{\lsun}{{\rm L}_\sun}$
$\newcommand{\kms}{km s^{-1}}$</div>



<div id="title">

# Digging into the Interior of Hot Cores with ALMA: Spiral Accretion into the High-mass Protostellar Core G336.01--0.82

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.18006-b31b1b.svg)](https://arxiv.org/abs/2311.18006)<mark>Appeared on: 2023-12-01</mark> -  _11 pages, 5 figures, 2 appendices. Accepted for publication in ApJL. Interactive figure available at this https URL_

</div>
<div id="authors">

F. A. Olguin, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** We observed the high-mass star-forming core G336.01--0.82 at 1.3 mm and 0 $\farcs$ 05 ( ${\sim}150$ au) angular resolution with the Atacama Large Millimeter/submillimeter Array (ALMA) as part of the Digging into the Interior of Hot Cores with ALMA (DIHCA) survey.These high-resolution observations reveal two spiral streamers feeding a circumstellar disk at opposite sides in great detail.Molecular line emission from $CH_3$ OH shows velocity gradients along the streamers consistent with infall.Similarly, a flattened envelope model with rotation and infall implies  a mass larger than  10 $\msun$ for the central source and a centrifugal barrier of 300 au.The location of the centrifugal barrier is consistent with local peaks in the continuum emission.We argue that gas brought by the spiral streamers is accumulating at the centrifugal barrier, which can result in future accretion burst events.A total high infall rate of ${\sim}4\times10^{-4}$ $\msun$ yr $^{-1}$ is derived by matching models to the observed velocity gradient along the streamers.Their contribution account for 20--50 \% the global infall rate of the core, indicating streamers play an important role in the formation of high-mass stars.

</div>

<div id="div_fig1">

<img src="tmp_2311.18006/./fig2.png" alt="Fig7" width="100%"/>

**Figure 7. -** $CH_3$OH $J_{K_a,K_c}=18_{3,15}-17_{4,14} A, v_t=0$ moment maps.
(a) shows the zeroth order moment map.
The blue and red lines mark the position of the north and south spiral-like features, respectively.
The arrow marks the direction of the pv map in Figure \ref{fig:ch3oh:pvmap}(a), and corresponds to the line crossing all three continuum structures (PA=125◦).
(b) shows the first order moment map, and (c) the second order moment map.
The continuum is shown in contours with the same levels as in Figure \ref{fig:continuum}, and the continuum peaks are marked with green triangles.
The beam size ellipse is shown in the lower left corner.
 (*fig:ch3oh:moment*)

</div>
<div id="div_fig2">

<img src="tmp_2311.18006/./fig1.png" alt="Fig6" width="100%"/>

**Figure 6. -** ALMA maps of G336.01--0.82.
(a) Continuum map at 1.3 mm.
The position of the three local continuum peaks is indicated by the green labeled triangles.
The contour levels are --3, 5, 10, 20, 40, $80\times\sigma_{\rm rms}$ with $\sigma_{\rm rms}=55$$\mu$Jy beam$^{-1}$.
(c) $HC_3$N $J=24-23 l=1f ,v_7=1$ zeroth moment map contours over the continuum map.
The contours are 5, 8, 10, 15, 20, $25\times\sigma_{\rm rms}$ with $\sigma_{\rm rms} = 11$ mJy beam$^{-1}$ km s$^{-1}$.
(b) SiO $J=5-4$ blue- and red-shifted, and
The contour levels are 5, 6, 7 ... $12\times\sigma_{\rm rms}$ with $\sigma_{\rm rms} = 7.7$ mJy beam$^{-1}$ km s$^{-1}$.
Velocity integration ranges with respect to the systemic velocity are annotated by the respective color.
The beam size of the continuum map is shown in the lower left corner.
 (*fig:continuum*)

</div>
<div id="div_fig3">

<img src="tmp_2311.18006/./fig3.png" alt="Fig8" width="100%"/>

**Figure 8. -** $CH_3$OH $J_{K_a,K_c}=18_{3,15}-17_{4,14} A, v_t=0$ pv maps.
(a) pv map following the gray line in Figure \ref{fig:ch3oh:moment}(a), i.e., perpendicular to the outflow/rotation axis (PA=125◦) and a slit width of 0$\farcs$1.
The dotted green lines show the offset of the continuum sources ALMA1-WEST (left) and ALMA1-EAST (right).
(b) pv map of the northern spiral-like feature following the blue line in Figure \ref{fig:ch3oh:moment}(a) and a slit width of 0$\farcs$05.
Zero offset corresponds to ALMA1-WEST.
(c) pv map of the southern spiral-like feature following the red line in Figure \ref{fig:ch3oh:moment}(a) and a slit width of 0$\farcs$05.
Gray contour levels are --6, --3, 3, 6, 12 and $24\times\sigma$ with $\sigma=1.2$ mJy beam$^{-1}$.
The blue contours in (a) are from an IRE model (see \S\ref{sec:discussion:rotation}) at the same levels as the data relative to the peak, while the blue lines in (b) and (c) correspond to linear fits to data over $10\sigma$.
Zero offset corresponds to ALMA1-EAST.
The dashed green line correspond to the systemic velocity $v_{\rm LSR}=-47.2$ km s$^{-1}$.
 (*fig:ch3oh:pvmap*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.18006"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\bycristina}[1]{\textcolor{red}{#1}}$
$\newcommand{\mireia}[1]{\textcolor{purple}{#1}}$
$\newcommand{\rhealucas}[1]{{\color{lightblue}#1}}$
$\newcommand{\amael}[1]{\textcolor{brown}{#1}}$
$\newcommand{\rossella}[1]{\textcolor{cobalt}{#1}}$
$\newcommand{\yb}[1]{\textcolor{orange}{#1}}$
$\newcommand{\gm}[1]{\textcolor{emerald}{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{MSUN}$</div>



<div id="title">

# Preparing for low surface brightness science with the Vera C. Rubin Observatory: A Comparison of Observable and Simulated Intracluster Light Fractions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.18016-b31b1b.svg)](https://arxiv.org/abs/2311.18016)<mark>Appeared on: 2023-12-01</mark> -  _Submitted for publication in MNRAS, posted to arXiv after responding to two positive rounds of referee comments. Key results in Figs 3, 5, 6 and 11_

</div>
<div id="authors">

S. Brough, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** Intracluster Light (ICL) provides an important record of the interactions galaxy clusters have undergone. However, we are limited in our understanding by our measurement methods. To address this we measure the fraction of cluster light that is held in the Brightest Cluster Galaxy and ICL (BCG+ICL fraction) and the ICL alone (ICL fraction) using observational methods (Surface Brightness Threshold-SB, Non-Parametric Measure-NP, Composite Models-CM, Multi-Galaxy Fitting-MGF) and new approaches under development (Wavelet Decomposition-WD) applied to mock images of 61 galaxy clusters ( $14<$ log $_{10}M_{200c}/M_{\odot}<14.5$ ) from four cosmological hydrodynamical simulations. We compare the BCG+ICL and ICL fractions from observational measures with those using simulated measures (aperture and kinematic separations). The ICL fractions measured by kinematic separation are significantly larger than observed fractions. We find the measurements are related and provide equations to estimate kinematic ICL fractions from observed fractions. The different observational techniques give consistent BCG+ICL and ICL fractions but are biased to underestimating the BCG+ICL and ICL fractions when compared with aperture simulation measures. Comparing the different methods and algorithms we find that the MGF algorithm is most consistent with the simulations, and CM and SB methods show the smallest projection effects for the BCG+ICL and ICL fractions respectively. The Ahad (CM), MGF and WD algorithms are best set up to process larger samples, however, the WD algorithm in its current form is susceptible to projection effects. We recommend that new algorithms using these methods are explored to analyse the massive samples that Rubin Observatory's Legacy Survey of Space and Time will provide.

</div>

<div id="div_fig1">

<img src="tmp_2311.18016/./Sims_BCG_clustermass.png" alt="Fig1.1" width="50%"/><img src="tmp_2311.18016/./Sims_clustermass.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** The simulated BCG+ICL mass (upper panel) and the simulated BCG+ICL fraction (middle panel) as a function of cluster mass. The lower panel shows the ICL fraction measured from the simulations in 2 different apertures (left panel: 30 kpc$-$1 Mpc; middle panel: 100 kpc$-$1 Mpc) and the right-hand panel shows the ICL fraction measured using kinematic separation (crosses) as a function of cluster mass. The different simulations are indicated by the legend and include the one cluster from the higher-resolution Magneticum Box4 simulation.
     (*fig:Sims_bcgmass*)

</div>
<div id="div_fig2">

<img src="tmp_2311.18016/./ICLBCG_Frac_clustermass.png" alt="Fig2.1" width="50%"/><img src="tmp_2311.18016/./ICLBCG_FracDiff_histo.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** 
    Observed BCG+ICL fraction (mean measurement over the measured projections). The upper panel shows the observed BCG+ICL fraction as a function of cluster halo mass coloured by measurement type for the 61 simulated clusters across the 4 simulations. We do not observe a dependence of any of the measures on cluster halo mass. The lower panel shows the difference between the Observed BCG+ICL fraction and the Simulated 0 - 1 Mpc Aperture measurement. The observed measurements are presented grouped by measurement type: Non-Parametric Measures (NP; Gonzalez and Martinez-Lombilla), Composite Models (CM; Ahad and Ragusa), Multi-Galaxy Fitting (MGF; CICLE) and Wavelet Decomposition (WD; DAWIS-SS and DAWIS-W). The Surface Brightness Threshold method is not included for BCG+ICL fractions as it removes the BCG by definition. The numbers of clusters measured are different for each of the observed measures.  This figure demonstrates that all methods agree to <0.1 dex in excising the contribution of satellites. (*fig:ICLBCG_Frac_clustermass*)

</div>
<div id="div_fig3">

<img src="tmp_2311.18016/./ICL_Frac_clustermass_aper.png" alt="Fig11" width="100%"/>

**Figure 11. -** Observed ICL fraction (mean measurement over simulated projections) as a function of cluster halo mass coloured by measurement type for the 61 simulated clusters across the 4 simulations. The different panels show the three Simulated aperture measures ($30-1$ Mpc, $50-1$ Mpc, $100-1$ Mpc). We do not observe a dependence of any of the measures on cluster halo mass. (*fig:ICL_Frac_clustermass_aper*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.18016"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\edit}[1]$</div>



<div id="title">

# A Two-Component Probability Distribution Function Describes the mid-IR Emission from the Disks of Star-Forming Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.18067-b31b1b.svg)](https://arxiv.org/abs/2311.18067)<mark>Appeared on: 2023-12-01</mark> -  _30 pages without appendix, 17 figures, (with appendix images of full sample: 56 pages, 39 figures), accepted in AJ_

</div>
<div id="authors">

D. Pathak, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** High-resolution JWST-MIRI images of nearby spiral galaxies reveal emission with complex substructures that trace dust heated both by massive young stars and the diffuse interstellar radiation field. We present high angular ( $0\farcs85$ ) and physical resolution ( $20{-}80$ pc) measurements of the probability distribution function (PDF) of mid-infrared (mid-IR) emission (7.7--2 $\mu$ m) from $19$ nearby star-forming galaxies from the PHANGS-JWST Cycle-1 Treasury. The PDFs of mid-IR emission from the disks of all 19 galaxies consistently show two distinct components: an approximately log-normal distribution at lower intensities and a high-intensity power-law component. These two components only emerge once individual star-forming regions are resolved. Comparing with locations of $\ion{H}{2}$ regions identified from VLT/MUSE H $\alpha$ -mapping, we infer that the power-law component arises from star-forming regions and thus primarily traces dust heated by young stars. In the continuum-dominated 21 $\mu$ m band, the power-law is more prominent and contains roughly half of the total flux. At 7.7--11.3 $\mu$ m, the power-law is suppressed by the destruction of small grains (including PAHs) close to $\ion{H}{2}$ regions while the log-normal component tracing the dust column in diffuse regions appears more prominent. The width and shape of the log-normal diffuse emission PDFs in galactic disks remain consistent across our sample, implying a log-normal gas column density $N$ (H) $\approx10^{21}$ cm $^{-2}$ shaped by supersonic turbulence with typical (isothermal) turbulent Mach numbers $\approx5-15$ . Finally, we describe how the PDFs of galactic disks are assembled from dusty $\ion{H}{2}$ regions and diffuse gas, and discuss how the measured PDF parameters correlate with global properties such as star-formation rate and gas surface density.

</div>

<div id="div_fig1">

<img src="tmp_2311.18067/./overalltiles_NGC4321.png" alt="Fig3" width="100%"/>

**Figure 3. -** Images and PDFs of emission at 7.7 $\mu m$, 10 $\mu m$, 11.3 $\mu m$, and 21 $\mu m$ for NGC4321 at $0$\farcs$85$(60 pc) resolution. **Top:** JWST images for NGC4321 in each MIRI filter, restricted to the footprint of JWST-MUSE coverage ($\sim 10$ kpc, dashed blue). **Middle:** PDFs of inclination-corrected intensity from each image. The colors in the PDF map to the colorbar in intensity used to display each image. Vertical dashed lines indicate the RMS noise level in each filter at $0$\farcs$85$. **Bottom:** The same PDFs as the middle row, now with the y-axis on a logarithmic stretch to highlight features at the high-intensity end. (*fig:overall_ngc4321*)

</div>
<div id="div_fig2">

<img src="tmp_2311.18067/./HII_PDFs_v0p9p2_atGauss0p85.png" alt="Fig8" width="100%"/>

**Figure 8. -** The galactic disk PDFs (in black) of our full sample of 19 galaxies at 7.7, 10, 11.3, and 21 $\mu$m show an overall log normal distribution with a power-law tail at higher intensities. The power-law component can be primarily attributed to emission from individual $\ion${H}{2} regions (blue) and the log normal component comes from emission from outside $\ion${H}{2} regions (red). Vertical dashed lines indicate the rms noise level in each filter for reference. The best-fit log normal mean $\mu$ and dispersion $\sigma$ are included for each PDF. (*fig:PDF_HII*)

</div>
<div id="div_fig3">

<img src="tmp_2311.18067/./HII_logPDFs_v0p9p2_atGauss0p85.png" alt="Fig9" width="100%"/>

**Figure 9. -** The disk PDFs (in black) of our sample of 19 galaxies at 7.7, 10, 11.3, and 21 $\mu$m decomposed by contributions from $\ion${H}{2} regions (blue) and outside (red). These are the same PDFs as Figure \ref{fig:PDF_HII} but in log-stretch to highlight the power-law component. The best-fit power-law is shown in dark blue. The corresponding best-fit $\ion${H}{2} region power-law index $m$ and log-log intercept $c$ for each PDF are included for each panel. (*fig:logPDF_HII*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.18067"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

330  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
